## Ingest lap_times folder

In [0]:
dbutils.widgets.text('p_data_source', '')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Step 1: Read lap_times files with manual schema

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
lap_times_schema = StructType([StructField('raceId', IntegerType(), False),
                             StructField('driverId', IntegerType(), True),
                             StructField('lap', IntegerType(), True),
                             StructField('position', IntegerType(), True),
                             StructField('time', StringType(), True),
                             StructField('milliseconds', IntegerType(), True)])


In [0]:
lap_times_df = spark.read \
            .schema(lap_times_schema)\
            .csv(f'{raw_folder_path}/lap_times')

#### Step 2: Rename columns

In [0]:

lap_times_renamed_df = lap_times_df.withColumnRenamed('raceId', 'race_id') \
                                .withColumnRenamed('driverId', 'driver_id')


#### Step 3: Add Ingestion Date

In [0]:
from pyspark.sql.functions import lit

lap_times_final_df = add_ingestion_date(lap_times_renamed_df.withColumn('data_source', lit(v_data_source)))

#### Step 4: Add data to data lake in parquet

In [0]:
lap_times_final_df.write\
                .mode('overwrite')\
                .format('parquet')\
                .saveAsTable('f1_processed.lap_times')


In [0]:
dbutils.notebook.exit("Success")